In [1]:
from __future__ import annotations # To prevent Subscript for class "list" will generate runtime exception; enclose type annotation in quotes
import seaborn as sns
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import scipy
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from Kmeans import *
from decimal import Decimal
from SVD import * 
%matplotlib inline
# pd.options.display.max_rows = 4000
# pd.options.display.max_columns = 4000

In [2]:
df = pd.read_csv('preprocessed.csv',index_col=0)
Genres=list(pd.read_csv('genre_order.csv',index_col=0)['0'])
df

,Gender,Age,Zip-code_1,Zip-code_2,Zip-code_3,Zip-code_4,Zip-code_5,Zip-code_6,Zip-code_7,Zip-code_8,...,Action,1960s,Animation,Comedy,Thriller,Mystery,1940s,Children's,Film-Noir,Drama
UserID,,,,,,,,,,,,,,,,,,,,,
1,0,0.017857,0,0,0,1,0,0,0,0,...,0.182449,0.148240,0.645186,0.504759,0.091603,7.379219e-02,0.114032,0.747438,0.004701,0.822336
2,1,1.000000,0,0,0,0,0,0,1,0,...,0.607944,0.114536,0.027614,0.273131,0.330404,2.202718e-02,0.017621,0.057179,0.004405,1.000000
3,1,0.446429,0,0,0,0,1,0,0,0,...,0.814808,0.061729,0.086422,1.000000,0.148147,6.255649e-07,0.154653,0.086422,0.125994,0.271601
4,1,0.803571,0,1,0,0,0,0,0,0,...,0.354289,0.078345,0.194632,1.000000,0.047685,1.704221e-01,0.144273,0.004760,0.104962,0.102874
5,1,0.446429,0,0,0,0,1,0,0,0,...,0.125002,0.000000,0.026040,0.346350,0.182293,3.906287e-02,0.005208,0.039062,0.018228,0.562499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0,0.446429,0,0,1,0,0,0,0,0,...,0.294317,0.111286,0.091950,0.552734,0.301040,7.549845e-02,0.051283,0.123925,0.046457,0.898209
6037,0,0.803571,0,0,0,0,0,0,1,0,...,0.257441,0.253624,0.003553,0.535784,0.650177,1.178472e-01,0.135690,0.049999,0.071433,1.000000
6038,0,1.000000,1,0,0,0,0,0,0,0,...,0.062499,0.000002,0.187502,1.000000,0.794578,2.552067e-01,0.403411,0.000000,0.134377,0.750100


In [3]:
num_cluster=26
to_remove=10
threshold=0

In [4]:
from SVD import * 
u,s,sigma,V_trans = ReducedSVD(df.to_numpy(),threshold,num_cluster)

In [5]:
df = pd.DataFrame(u@sigma@V_trans,columns=df.columns)

In [6]:
def cluster_to_matrix(X_train,y_train,num_cluster):
    k=Kmeans(K=num_cluster,iter=150,p=2,random_state=42)
    y_pred,_=k.predict(np.array(X_train),choice=0)
    k.centroids=np.array(k.centroids)
    empty_cluster = []
    for idx,i in enumerate(k.clusters):
        if len(i)==0:
            empty_cluster.append(idx)
    X_train2 = pd.concat([X_train,y_train,pd.DataFrame(y_pred,columns=["label"],dtype='int')],axis="columns")
    df_cluster_rep = X_train2.groupby(['label'], as_index=False).mean()
    df_cluster_rep.drop(labels="label",axis="columns",inplace=True)
    matrix = df_cluster_rep.to_numpy()
    final_matrix = []
    j = 0 
    for i in range(num_cluster):
        if i in empty_cluster:
            final_matrix.append(np.zeros(len(df.columns)))
        else:
            final_matrix.append(matrix[j])
            j+=1

    return np.array(final_matrix),k 

In [7]:
x_col_size=len(df.columns)-len(Genres)+6

In [8]:
x_col_size

38

In [9]:
y = df[df.columns[x_col_size:]]
x = df.drop(labels=df.columns[x_col_size:],axis="columns")

In [10]:
x

,Gender,Age,Zip-code_1,Zip-code_2,Zip-code_3,Zip-code_4,Zip-code_5,Zip-code_6,Zip-code_7,Zip-code_8,...,Occupation_17,Occupation_18,Occupation_19,Occupation_20,Documentary,Sci-Fi,Musical,1910s,War,1970s
0,-0.010305,-0.095526,-0.049912,0.010171,-0.056252,0.953359,-0.006687,-0.047013,0.009295,-0.026782,...,-0.027998,-0.008609,0.029695,-0.024458,0.052312,0.095871,0.359506,0.010575,0.136174,0.088800
1,0.958074,0.909879,-0.038295,0.048108,-0.042834,-0.038450,0.019378,-0.029085,1.040172,0.003555,...,-0.018354,0.015433,0.029572,-0.007215,-0.000404,0.176359,-0.008925,0.010794,0.204333,0.128087
2,0.990069,0.366524,-0.039884,0.001624,-0.061096,-0.060083,0.983485,-0.035035,-0.006257,-0.027666,...,-0.054572,0.091164,0.079101,-0.047276,0.056396,0.285421,0.124182,-0.004406,0.311360,0.209543
3,1.048466,0.813215,0.026817,0.945200,0.024408,0.026056,-0.025569,0.020890,-0.047868,-0.009741,...,0.006712,-0.004973,-0.048671,-0.011188,0.080906,0.111669,0.127534,0.017626,0.034278,0.206274
4,0.956961,0.399148,-0.072420,0.025996,-0.078395,-0.072985,0.990618,-0.064904,0.016156,-0.028158,...,-0.035526,0.034469,0.048621,0.974993,0.009678,0.073449,-0.003652,0.010294,0.043920,-0.006701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,-0.039210,0.372349,-0.070851,0.010983,0.901566,-0.093663,-0.025710,-0.059718,-0.005503,-0.035571,...,-0.055947,0.051870,0.090331,-0.052181,0.008146,0.286535,0.060124,-0.004742,0.242785,0.172896
6036,-0.062136,0.730200,-0.082833,0.043811,-0.084111,-0.080148,0.000051,-0.064262,1.031082,-0.023311,...,-0.036773,0.001919,0.034309,-0.013485,0.041822,0.372178,0.027477,0.010287,0.167354,0.400697
6037,-0.001930,0.917685,1.012029,0.021758,0.012854,0.016918,0.028892,0.011854,0.019658,0.018406,...,0.009502,-0.001075,0.002240,0.004311,0.136240,0.082816,0.246865,0.021417,0.135408,0.179050
6038,-0.008366,0.802050,0.994133,-0.003608,-0.000752,0.000056,0.001381,0.004014,-0.002706,0.001132,...,-0.003192,-0.014546,-0.015712,0.002879,0.133102,0.132944,0.305340,0.020980,0.159652,0.324635


In [11]:
y

,Western,Horror,1980s,1950s,1990s,Crime,Romance,Fantasy,2000s,Adventure,...,Action,1960s,Animation,Comedy,Thriller,Mystery,1940s,Children's,Film-Noir,Drama
0,0.023201,0.255289,0.219771,0.046293,1.037540,0.039547,0.289658,0.241559,0.223732,0.295636,...,0.189872,0.045687,0.575855,0.592819,0.108639,0.040326,0.031920,0.819769,0.046727,0.929704
1,0.008585,-0.018751,0.247560,0.028082,1.036941,0.076464,0.309087,-0.009027,-0.027116,0.261455,...,0.575223,0.096429,0.022578,0.304969,0.341180,0.031771,0.015733,0.036215,0.050330,1.059812
2,0.133627,-0.033900,0.763793,0.128459,0.879182,0.337496,0.172517,0.159600,-0.121756,0.573137,...,0.861768,0.237991,0.041926,1.010600,0.147686,0.129016,0.145362,0.117945,0.104867,0.255064
3,0.096529,-0.066917,0.294882,0.193681,0.094342,0.036061,0.060110,0.035995,0.237514,0.096101,...,0.240533,0.162260,0.095341,0.845253,0.088052,0.102623,0.181631,0.115924,0.087706,-0.058689
4,-0.007495,0.046491,-0.035602,-0.038134,1.109797,0.068655,0.184995,0.016872,-0.021433,0.089509,...,0.110614,0.025206,0.013714,0.420176,0.222625,-0.003542,-0.013293,0.021351,0.037046,0.661581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.032120,0.147192,0.418012,0.051928,1.171025,0.302074,0.245451,0.126934,-0.171939,0.279539,...,0.246347,0.160377,0.017281,0.611667,0.228082,0.092426,0.053635,0.077697,0.069073,0.950969
6036,0.013423,0.126119,0.308322,0.069430,0.671328,0.235552,0.267915,0.025253,0.079883,0.122371,...,0.287902,0.144283,-0.005070,0.597427,0.517888,0.143386,0.085319,-0.034207,0.147137,1.112842
6037,0.123527,0.053428,0.430712,0.269936,0.573055,0.474686,0.551995,0.052970,0.313681,-0.048529,...,0.144262,0.195627,0.131595,1.001394,0.697589,0.338731,0.302043,0.142782,0.278511,0.789987
6038,0.131770,-0.013879,0.264254,0.310120,0.347824,0.166226,0.535320,0.124406,0.237870,0.167822,...,0.105424,0.317751,0.218463,0.944657,0.135816,0.197328,0.279735,0.297022,0.185692,0.422159


In [12]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.05, shuffle=True, random_state=42)

In [13]:
matrix,kmeans_model = cluster_to_matrix(x_train,y_train,num_cluster)

In [14]:
matrix.shape

(25, 60)

In [15]:
def predict(datapoint,kmeans,matrix_clean):
    ''' 
        @input: Takes in a datapoint, kmeans object, and information matrix 
        @output: Returns: rating sorted with index for column name mapping
    '''
    y_pred = kmeans.predictPoint(datapoint)
    inferred_full = matrix_clean[y_pred]
    inferred = inferred_full[x_col_size:]
    inferred_full = inferred_full[len(df.columns)-len(Genres):]
    sorted_ratings = []
    sorted_ratings_full = []
    for idx,element in enumerate(inferred):
        sorted_ratings.append([element,idx+x_col_size])
    for idx,element in enumerate(inferred_full):
        sorted_ratings_full.append([element,idx+len(df.columns)-len(Genres)])
    sorted_ratings.sort()
    sorted_ratings.reverse()
    sorted_ratings_full.sort()
    sorted_ratings_full.reverse()
    return sorted_ratings, sorted_ratings_full

In [16]:
x_test.to_numpy()[0][:x_col_size]
sorted_ratings, sorted_ratings_full = predict(x_test.to_numpy()[0][:x_col_size],kmeans_model,matrix)
s="Top 5 PREDICTED recommended genres are:\n"
for i in range(5):
    s+=df.columns[sorted_ratings_full[i][1]]+"\n"
print(s)

Top 5 PREDICTED recommended genres are:
1990s
Drama
Comedy
Action
Thriller



In [17]:
sorted_ratings_actual=[]
inferred = y_test.to_numpy()[0]
for idx,element in enumerate(inferred):
    sorted_ratings_actual.append([element,idx+x_col_size])
sorted_ratings_actual.sort()
sorted_ratings_actual.reverse()
s="Top 5 ACTUAL recommended genres are:\n"
for i in range(5):
    s+=df.columns[sorted_ratings_actual[i][1]]+"\n"
print(s)

Top 5 ACTUAL recommended genres are:
1990s
Drama
Comedy
Romance
1980s



In [18]:
rating_dict={}
for i in range(len(sorted_ratings_full)):
    rating_dict[df.columns[sorted_ratings_full[i][1]]]=sorted_ratings_full[i][0]
rating_dict

{'1990s': 0.8360713808179497,
 'Drama': 0.6493539645174066,
 'Comedy': 0.5987218463843071,
 'Action': 0.4176737365943115,
 'Thriller': 0.31688192582398905,
 '1980s': 0.3167629457391918,
 'Sci-Fi': 0.2785316834677667,
 'Romance': 0.26308192338225767,
 'Adventure': 0.25459672733595773,
 '1970s': 0.21017458946028994,
 '1960s': 0.15735952288562316,
 "Children's": 0.15712309006292022,
 'Crime': 0.15617862932427432,
 'Horror': 0.1516362618054006,
 'War': 0.15108542861218405,
 'Animation': 0.12583702645652603,
 'Musical': 0.11245858017082506,
 'Fantasy': 0.11153733154134934,
 '2000s': 0.10823102449164324,
 'Film-Noir': 0.1055710461269595,
 '1940s': 0.10449845882781346,
 'Mystery': 0.10382833459564583,
 '1950s': 0.10066956017795994,
 '1930s': 0.07032707521663636,
 'Documentary': 0.06543768453796635,
 'Western': 0.058251166399954354,
 '1920s': 0.052619294541720735,
 '1910s': 0.012536626361596965}

In [19]:
def predict_for_dataset(X_test,kmeans_instance,matrix_clean):
    X_test = X_test.to_numpy()
    y_pred = []
    for i in range(X_test.shape[0]):
        datapoint = X_test[i]
        y_pred.append(predict(datapoint,kmeans_instance,matrix_clean)[0])
    return np.array(y_pred)

In [20]:
x_test

,Gender,Age,Zip-code_1,Zip-code_2,Zip-code_3,Zip-code_4,Zip-code_5,Zip-code_6,Zip-code_7,Zip-code_8,...,Occupation_17,Occupation_18,Occupation_19,Occupation_20,Documentary,Sci-Fi,Musical,1910s,War,1970s
5529,-0.095286,0.210652,-0.135276,0.080550,-0.143108,-0.132784,0.006844,0.883178,0.061877,-0.038701,...,-0.063250,0.008968,0.072207,-0.031492,-0.005960,0.087990,0.001645,0.014364,0.072035,0.074900
710,0.949837,0.457729,-0.054222,0.034845,-0.062327,-0.062074,0.012794,-0.035898,0.021365,0.985417,...,0.012692,0.033181,0.049658,0.044994,0.012310,0.222733,0.019622,0.005671,0.185678,0.435713
4923,-0.075279,0.503694,-0.119979,0.061965,-0.132852,-0.121391,-0.000767,-0.106386,0.045318,-0.038535,...,-0.053615,0.007357,0.052684,-0.032368,0.064016,0.016262,0.018084,0.018959,0.117366,0.083897
2153,1.002609,0.426415,-0.002713,0.007621,0.005123,0.000360,0.004014,0.996017,0.005729,0.008090,...,0.002591,0.005844,0.016686,-0.002502,0.094673,0.270057,0.156440,0.019006,0.180912,0.334225
1272,0.987756,0.631553,0.997394,0.017991,0.000737,-0.000826,0.011656,0.004563,0.017373,0.005814,...,-0.006749,0.015541,0.024706,-0.001693,0.048133,0.103249,0.070981,0.012194,0.140247,0.321897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3341,-0.088645,0.381805,-0.116444,0.031278,-0.122888,-0.099459,-0.012377,-0.095848,0.012896,-0.037410,...,-0.024484,0.041863,0.078448,0.010298,0.091592,0.139068,0.000407,0.015737,0.020838,-0.023315
4648,1.022920,1.180224,1.004021,-0.013587,0.043800,0.018327,0.000387,0.003587,-0.001974,-0.008180,...,-0.006113,0.044721,-0.004717,-0.015036,0.051938,0.083845,0.141343,0.018132,0.278692,0.161845
323,0.969677,0.652832,0.005137,0.017666,0.009472,0.006873,1.018534,0.017838,0.015700,0.017091,...,1.007519,0.017697,0.000610,0.020567,0.099209,0.240172,0.238586,0.015925,0.141227,0.084766
2943,-0.078498,0.263714,-0.081003,0.073614,-0.095827,0.909729,0.018988,-0.067280,0.058897,-0.006999,...,-0.035336,-0.023074,0.031976,-0.017308,0.047388,0.274928,0.228084,0.016517,0.099735,0.135402


In [21]:
y_pred = predict_for_dataset(x_test,kmeans_model,matrix) 

In [22]:
y_pred=np.array(y_pred)
y_test=np.array(y_test)

In [23]:
y_test.shape

(302, 22)

In [24]:
y_pred_arr=[]
for i in y_pred:
    temp=[0 for _ in range(y_test.shape[1])]
    for j in i:
        temp[int(j[1]-x_col_size)]=j[0]
    y_pred_arr.append(temp)
y_pred_arr=np.array(y_pred_arr)
y_pred_arr.shape


(302, 22)

In [25]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred_arr)

0.03286051572030125

In [26]:
ratings=pd.read_csv('ml-1m/ratings.dat', delimiter='::', names=['UserID','MovieID','Rating','TimeStamp'])
movies=pd.read_csv('ml-1m/movies.dat', delimiter='::', names=["Title","Genres"],  encoding="ISO-8859-1")

/tmp/ipykernel_20564/2496518303.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings=pd.read_csv('ml-1m/ratings.dat', delimiter='::', names=['UserID','MovieID','Rating','TimeStamp'])
/tmp/ipykernel_20564/2496518303.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies=pd.read_csv('ml-1m/movies.dat', delimiter='::', names=["Title","Genres"],  encoding="ISO-8859-1")


In [27]:
# movies
ratings.drop(['UserID','TimeStamp'], axis=1, inplace= True)

In [28]:
ratings

,MovieID,Rating
0,1193,5
1,661,3
2,914,3
3,3408,4
4,2355,5
...,...,...
1000204,1091,1
1000205,1094,5
1000206,562,5
1000207,1096,4


In [29]:
movies['Rating']=[[] for _ in range(len(movies.index))]
for i in ratings.index:
    temp=ratings.iloc[i]['Rating']
    movies.at[ratings.iloc[i]['MovieID'],'Rating'].append(temp)

In [30]:
movies['Rating']=movies['Rating'].apply(lambda x: np.array(x).mean())

/tmp/ipykernel_20564/1822539475.py:1: RuntimeWarning: Mean of empty slice.
  movies['Rating']=movies['Rating'].apply(lambda x: np.array(x).mean())
/home/netradeepakc/anaconda3/envs/rec_sys_mini/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [31]:
movies.isna().sum()

Title       0
Genres      0
Rating    177
dtype: int64

In [32]:
movies['Genres']=movies['Genres'].apply(lambda x: x.split('|'))
for i in movies.index:
    title=movies.at[i,'Title']
    decade=title[-5:-2]
    decade=str(decade)+'0s'
    movies.at[i,'Genres'].append(decade)

In [33]:
movies

,Title,Genres,Rating
1,Toy Story (1995),"[Animation, Children's, Comedy, 1990s]",4.146846
2,Jumanji (1995),"[Adventure, Children's, Fantasy, 1990s]",3.201141
3,Grumpier Old Men (1995),"[Comedy, Romance, 1990s]",3.016736
4,Waiting to Exhale (1995),"[Comedy, Drama, 1990s]",2.729412
5,Father of the Bride Part II (1995),"[Comedy, 1990s]",3.006757
...,...,...,...
3948,Meet the Parents (2000),"[Comedy, 2000s]",3.635731
3949,Requiem for a Dream (2000),"[Drama, 2000s]",4.115132
3950,Tigerland (2000),"[Drama, 2000s]",3.666667
3951,Two Family House (2000),"[Drama, 2000s]",3.900000


In [34]:
movies['Affinity']=[0 for _ in range(len(movies.index))]
for i in movies.index:
    genres=movies.at[i,'Genres']
    affinity=0
    for j in genres:
        affinity+=rating_dict[j]*movies.at[i,'Rating']
    movies.at[i,'Affinity']=affinity

/tmp/ipykernel_20564/3621030468.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '7.12325929199568' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  movies.at[i,'Affinity']=affinity


In [35]:
movies.sort_values('Affinity',inplace=True,ascending=False)

In [36]:
movies

,Title,Genres,Rating,Affinity
3266,Man Bites Dog (C'est arrivé près de chez vous)...,"[Action, Comedy, Crime, Drama, 1990s]",3.655844,9.717232
1215,Army of Darkness (1993),"[Action, Adventure, Comedy, Horror, Sci-Fi, 19...",3.804233,9.652220
293,"Professional, The (a.k.a. Leon: The Profession...","[Crime, Drama, Romance, Thriller, 1990s]",4.106176,9.122147
838,Emma (1996),"[Comedy, Drama, Romance, 1990s]",3.880240,9.107811
21,Get Shorty (1995),"[Action, Comedy, Drama, 1990s]",3.623894,9.066333
...,...,...,...,...
3650,Anguish (Angustia) (1986),"[Horror, 1980s]",NaN,NaN
3750,Boricua's Bond (2000),"[Drama, 2000s]",NaN,NaN
3829,Mad About Mambo (2000),"[Comedy, Romance, 2000s]",NaN,NaN
3856,Autumn Heart (1999),"[Drama, 1990s]",NaN,NaN


In [37]:
top_movies=list(movies['Title'])[:5]
print("Top 5 movies are:")
for i in top_movies:
    print(i)

Top 5 movies are:
Man Bites Dog (C'est arrivé près de chez vous) (1992)
Army of Darkness (1993)
Professional, The (a.k.a. Leon: The Professional) (1994)
Emma (1996)
Get Shorty (1995)
